# Import used libraries

In [1]:
# !pip3 install 'selenium<4.0.0'

In [2]:
from config import *
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import os
# import pymongo
from time import sleep
import pandas as pd
import numpy as np
import re
import csv
import sys
from time import sleep
from multiprocessing import Process
from selenium.webdriver.common.by import By

## Connect to mongodb

you should have your own parameters of: 
- mongo user
- mongo pass
- mongo_url

apply these three paramters in config file.

**I have separate the positive reviews from negative reviews in two databases.**

In [3]:
# client = pymongo.MongoClient(f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_url}")
# db1 = client.positive_reviews
# client = pymongo.MongoClient(f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_url}")
# db2 = client.negative_reviews

In [4]:
try:
    current_path = os.path.dirname(os.path.abspath(__file__))
except:
    current_path = '.'

In [5]:
def init_driver(gecko_driver='', user_agent='', load_images=True, is_headless=False):
    '''
        This function is just to set up some of default for browser
    '''
    firefox_profile = webdriver.FirefoxProfile()
    
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)
    firefox_profile.set_preference("media.volume_scale", "0.0")
    firefox_profile.set_preference("dom.webnotifications.enabled", False)
    if user_agent != '':
        firefox_profile.set_preference("general.useragent.override", user_agent)
    if not load_images:
        firefox_profile.set_preference('permissions.default.image', 2)

    options = Options()
    options.headless = is_headless
    
    driver = webdriver.Firefox(options=options,
                               executable_path=f'{current_path}/{gecko_driver}',
                               firefox_profile=firefox_profile)
    
    return driver

In [6]:
def get_url(url, driver):
    '''
    Argument:
        url of any page to get
        driver that was inilized
    return:
        True
    '''
    driver.get(url)
    driver.refresh()
    return True


### Call the main function with multithreading process

first of all we launch our driver and then click on the require languages we need to translate "from Engish" and "to Arabic".

so you will see that en_selctor is the source language and ar_selctor that we translate to.

I just use condition that present each 3000 translated reviews.

for each time we send English review to google we need to ensure that its cleared once translated to Arabic.

based on using multiprocessing that help us translate positive and negative reviews at the same time we enshure that the sourse is positive or negative reviews and then send to database once it has no review like that because of repated entry in any time we translate other reviews.

close the driver once it finish.

In [7]:
def automated_google_translation(list_of_reviews, url):
    driver = init_driver(gecko_driver,user_agent=user_agent)
    get_url(url, driver)
    en_selctor = driver.find_element_by_css_selector('#i8')
    en_selctor.click()
    ar_selctor = driver.find_element_by_css_selector('#i12')
    ar_selctor.click()
    
    click_out = driver.find_element_by_css_selector('.hgbeOc.EjH7wc').click()
    
    translated_to_en = []
#     translated_reviews_negative = []
    
    for i, reviews in enumerate(list_of_reviews):
        if i % 10000 == 0:
            print("Translate " + str(i) + " reviews")
            
        source = driver.find_element_by_css_selector('textarea.er8xn')
        source.clear()
        source.send_keys(reviews)
        sleep(1)
        translate = driver.find_element_by_css_selector('.dePhmb .eyKpYb .Q4iAWc')
        translated_to_en.append(translate.text)
#         print(translate.text)
#         break
        
    driver.close()
    return translated_to_en

## Reading csv
you need to read you file from csv_files the same file has classifed Arabic reviews with 5000 review.

so for another time you have another file with english reviews just apply it in the dir.

you should apply your column that contain the polarity of the review instead of **'reviews.rating'**

In [8]:
df = pd.read_csv('all_poems_w_clean verse_no_missing_drop.csv')

In [9]:
df.head()

,Unnamed: 0,period,poet,rhyme_scheme,meter,clean_verse
0,0,قبل الإسلام,عمرو بنِ قُمَيئَة,د,الطويل,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَ...
1,1,قبل الإسلام,عمرو بنِ قُمَيئَة,د,الطويل,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُرعَ...
2,2,قبل الإسلام,عمرو بنِ قُمَيئَة,د,الطويل,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَ...
3,3,قبل الإسلام,عمرو بنِ قُمَيئَة,د,الطويل,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُني ...
4,4,قبل الإسلام,عمرو بنِ قُمَيئَة,د,الطويل,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ...


In [10]:
df.dropna(inplace=True)

In [11]:
clean_verse = list(df['clean_verse'])
# list_of_negative = df[df['reviews.rating'] == 0]

In [12]:
len(clean_verse)

1684668

In [13]:
TASHKEEL           = re.compile(r'[\u0617-\u061A\u064B-\u0652]')

In [14]:
def remove_taskel(text):
    return TASHKEEL.sub(r"", text)

In [15]:
clean_verse = [remove_taskel(text) for text in clean_verse]
clean_verse[:3]

['خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وتنتظرا غدا',
 'فما لبث يوما بسابق مغنم ولا سرعتي يوما بسابقة الردى',
 'وإن تنظراني اليوم أقض لبانة وتستوجبا منا علي وتحمدا']

In [16]:
# list_of_positive_reviews_english = list(list_of_positive['reviews.text'])
# list_of_negative_reviews_english = list(list_of_negative['reviews.text'])

In [ ]:
# print(len(list_of_positive_reviews_english))
# len(list_of_negative_reviews_english)

In [17]:
translated_to_en = automated_google_translation(clean_verse[:50], google_translate_url)

Translate 0 reviews
Translate 10 reviews
Translate 20 reviews
Translate 30 reviews
Translate 40 reviews


In [18]:
# translated_to_en

['Khalili, do not rush to provide a comprehensive gathering and wait tomorrow',
 'So soon I am a former sheep, nor my speed, one day in the precedent',
 'And if you look at me today, I will spend a bila and you will be obligated to me and praise us',
 'For your age, the same is hard, rational, I am a secret of my secret, to get angry',
 'And if it appears from him a lot and branches in Lumi again and ascending',
 'I have to be guilty to be his fairy except the saying of Bagh',
 'For my life, for a person who calls for his rope if he calls in the maqam',
 'Great ashes of destiny, not tired, or ethnic',
 'And if you declare a eyeliner, a garden from the wind did not leave the money.',
 'I have been patient with the loyalty of the loyalty and destroy them if they are close to them',
 'Faraj Al -Hayy did not protect except the governor of Karim Al -Muhaya, Majid, other than the one',
 'I see my neighbor, I was afraid, her advice and love for her had it not been for the nuclei and its ambit

In [ ]:
# if __name__ == '__main__':
#     p1 = Process(target=automated_google_translation, 
#                  args=(list_of_positive_reviews_english,google_translate_url,'positive'))
# #     p1.start()
# #     p2 = Process(target=automated_google_translation, 
# #                  args=(list_of_negative_reviews_english,google_translate_url, 'negative'))
#     p2.start()
#     p1.join()
#     p2.join()

![alt text](images/negative.png "negative_reviews")
![alt text](images/positive.png "positive_reviews")